# 🤖 Claude Android Controller - Colab Client

This notebook allows Claude to connect to a GitHub Actions-hosted Android emulator.

## Prerequisites
1. GitHub Actions workflow is running (see README.md)
2. Copy the ngrok URL from the workflow output

## Quick Start
1. Run all cells in order
2. Enter the ngrok URL when prompted
3. Use the executor to control the Android emulator

In [ ]:
# Cell 1: Install dependencies
!pip install httpx frida-tools objection -q
print("✅ Dependencies installed")

In [ ]:
# Cell 2: Android Remote Executor Client

import httpx
from typing import Optional, Dict, Any
from pathlib import Path

class AndroidRemoteExecutor:
    """Client for connecting to Android + Frida Remote Executor"""
    
    def __init__(self, base_url: str, timeout: int = 300):
        self.base_url = base_url.rstrip('/')
        self.timeout = timeout
        self.client = httpx.Client(timeout=timeout)
        
    def _request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        url = f"{self.base_url}{endpoint}"
        try:
            response = self.client.request(method, url, **kwargs)
            return response.json()
        except httpx.TimeoutException:
            return {"success": False, "error": "Request timed out"}
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    # System
    def health(self): return self._request("GET", "/health")
    def info(self): return self._request("GET", "/")
    def disk(self): return self._request("GET", "/disk")
    
    # Execution
    def execute(self, code: str, timeout: int = None):
        payload = {"code": code}
        if timeout: payload["timeout"] = timeout
        return self._request("POST", "/execute", json=payload)
    
    def bash(self, command: str, timeout: int = None):
        payload = {"command": command}
        if timeout: payload["timeout"] = timeout
        return self._request("POST", "/bash", json=payload)
    
    # ADB
    def adb(self, command: str, timeout: int = None):
        payload = {"command": command}
        if timeout: payload["timeout"] = timeout
        return self._request("POST", "/adb", json=payload)
    
    def adb_devices(self): return self._request("GET", "/adb/devices")
    def adb_packages(self, filter: str = None):
        params = {"filter": filter} if filter else {}
        return self._request("GET", "/adb/packages", params=params)
    
    # Frida
    def frida_connect(self): return self._request("POST", "/frida/connect")
    def frida_processes(self): return self._request("GET", "/frida/processes")
    def frida_attach(self, process: str = "Gadget"):
        return self._request("POST", f"/frida/attach?process_name={process}")
    def frida_script(self, script: str, process: str = "Gadget", name: str = "default"):
        return self._request("POST", "/frida/script", json={"script": script, "process_name": process, "script_name": name})
    def frida_ssl_bypass(self, process: str = "Gadget"):
        return self._request("POST", f"/frida/ssl-bypass?process_name={process}")
    def frida_network_monitor(self, process: str = "Gadget"):
        return self._request("POST", f"/frida/network-monitor?process_name={process}")
    def frida_scripts(self): return self._request("GET", "/frida/scripts")
    def frida_unload(self, name: str): return self._request("DELETE", f"/frida/script/{name}")
    
    # APK
    def apk_patch(self, path: str, arch: str = "x86_64"):
        return self._request("POST", "/apk/patch", json={"apk_path": path, "architecture": arch})
    def apk_install(self, path: str):
        return self._request("POST", f"/apk/install?apk_path={path}")
    def apk_info(self, path: str):
        return self._request("GET", f"/apk/info?apk_path={path}")
    def apk_launch(self, package: str):
        return self._request("POST", f"/apk/launch?package_name={package}")
    
    # Files
    def download(self, url: str, dest: str = "/tmp/downloads", filename: str = None):
        payload = {"url": url, "destination": dest}
        if filename: payload["filename"] = filename
        return self._request("POST", "/download", json=payload)
    def ls(self, path: str = "/tmp"): return self._request("GET", "/ls", params={"path": path})
    def read(self, path: str): return self._request("GET", "/read", params={"path": path})
    
    # Convenience
    def full_setup(self, apk_url: str):
        """Download, patch, install, and launch an APK"""
        import time
        results = {}
        
        print("📥 Downloading APK...")
        results['download'] = self.download(apk_url, "/tmp/apks")
        if not results['download'].get('success'): return results
        apk_path = results['download']['filepath']
        
        print("📋 Getting APK info...")
        results['info'] = self.apk_info(apk_path)
        pkg = results['info'].get('info', {}).get('package_name')
        
        print("🔧 Patching APK...")
        results['patch'] = self.apk_patch(apk_path)
        if not results['patch'].get('success'): return results
        patched = results['patch']['patched_apk']
        
        print("📲 Installing APK...")
        results['install'] = self.apk_install(patched)
        
        if pkg:
            print(f"🚀 Launching {pkg}...")
            results['launch'] = self.apk_launch(pkg)
            time.sleep(3)
        
        print("🔌 Connecting Frida...")
        results['frida'] = self.frida_connect()
        
        print("✅ Setup complete!")
        results['success'] = True
        results['package_name'] = pkg
        return results

print("✅ AndroidRemoteExecutor class defined")

In [ ]:
# Cell 3: Connect to GitHub Actions
#@title 📱 Connect to Android Remote Executor
#@markdown Enter the ngrok URL from GitHub Actions output:

NGROK_URL = "https://xxxx-xx-xx-xxx-xx.ngrok-free.app"  #@param {type:"string"}

executor = AndroidRemoteExecutor(NGROK_URL)

# Test connection
health = executor.health()
if health.get('status') == 'healthy':
    print("✅ Connected to Android Remote Executor!")
    print(f"   CPU: {health.get('cpu_percent')}%")
    print(f"   Memory: {health.get('memory_percent')}%")
    print(f"   Disk Free: {health.get('disk_free_gb')} GB")
    print(f"   Emulator: {'✅' if health.get('emulator_connected') else '❌'}")
    print(f"   Frida: v{health.get('frida_version')}")
else:
    print(f"❌ Connection failed: {health.get('error')}")

In [ ]:
# Cell 4: List Available Devices
print("📱 ADB Devices:")
devices = executor.adb_devices()
for d in devices.get('devices', []):
    print(f"   {d['id']} - {d['state']} {d.get('info', '')}")

In [ ]:
# Cell 5: Download and Setup APK
#@title 📦 Download and Setup APK
#@markdown Enter the URL of the APK to analyze:

APK_URL = "https://example.com/app.apk"  #@param {type:"string"}

# Full automated setup
result = executor.full_setup(APK_URL)

if result.get('success'):
    print(f"\n🎉 Ready to analyze: {result.get('package_name')}")
else:
    print(f"\n❌ Setup failed: {result}")

In [ ]:
# Cell 6: Bypass SSL Pinning
#@title 🔓 Bypass SSL Pinning

result = executor.frida_ssl_bypass()
if result.get('success'):
    print("✅ SSL Pinning bypass active!")
    print("   Now intercept HTTPS traffic with your proxy.")
else:
    print(f"❌ Failed: {result.get('error')}")

In [ ]:
# Cell 7: Monitor Network Traffic
#@title 📡 Monitor Network Traffic

result = executor.frida_network_monitor()
if result.get('success'):
    print("✅ Network monitoring active!")
    print("   Interact with the app to see traffic...")
else:
    print(f"❌ Failed: {result.get('error')}")

In [ ]:
# Cell 8: Custom Frida Script
#@title 📝 Load Custom Frida Script
#@markdown Write your Frida JavaScript code:

FRIDA_SCRIPT = """
Java.perform(function() {
    console.log("[*] Custom script loaded!");
    
    // Example: Hook SharedPreferences
    var SharedPreferences = Java.use('android.content.SharedPreferences');
    var Editor = Java.use('android.content.SharedPreferences$Editor');
    
    Editor.putString.implementation = function(key, value) {
        console.log("[SharedPrefs] PUT " + key + " = " + value);
        return this.putString(key, value);
    };
    
    console.log("[+] SharedPreferences hook installed");
});
"""  #@param {type:"raw"}

result = executor.frida_script(FRIDA_SCRIPT, name="custom")
if result.get('success'):
    print("✅ Script loaded!")
    if result.get('initial_messages'):
        print("📨 Messages:")
        for msg in result['initial_messages']:
            print(f"   {msg}")
else:
    print(f"❌ Failed: {result.get('error')}")

In [ ]:
# Cell 9: Execute ADB Commands
#@title 🔧 Execute ADB Command

ADB_COMMAND = "shell pm list packages"  #@param {type:"string"}

result = executor.adb(ADB_COMMAND)
if result.get('success'):
    print(result.get('stdout', ''))
else:
    print(f"❌ Error: {result.get('stderr', result.get('error'))}")

In [ ]:
# Cell 10: Execute Python Code on Remote
#@title 🐍 Execute Python Code on Remote Server

PYTHON_CODE = """
import os
print(f"Working directory: {os.getcwd()}")
print(f"Files: {os.listdir('/tmp')}")
"""  #@param {type:"raw"}

result = executor.execute(PYTHON_CODE)
if result.get('success'):
    print(result.get('stdout', ''))
    if result.get('result'):
        print(f"Result: {result['result']}")
else:
    print(f"❌ Error: {result.get('error')}")

In [ ]:
# Cell 11: List Files on Remote

PATH = "/tmp/apks"  #@param {type:"string"}

result = executor.ls(PATH)
if result.get('success'):
    print(f"📂 {result['path']}:")
    for item in result.get('items', []):
        icon = "📁" if item['is_dir'] else "📄"
        size = f" ({item['size']} bytes)" if item.get('size') else ""
        print(f"   {icon} {item['name']}{size}")
else:
    print(f"❌ Error: {result.get('error')}")

In [ ]:
# Cell 12: List Loaded Frida Scripts

result = executor.frida_scripts()
if result.get('success'):
    scripts = result.get('scripts', [])
    if scripts:
        print("📜 Loaded Frida scripts:")
        for s in scripts:
            print(f"   • {s}")
    else:
        print("No scripts loaded.")
else:
    print(f"❌ Error: {result.get('error')}")

---

## 📚 Quick Reference

| Method | Description |
|--------|-------------|
| `executor.health()` | Check system status |
| `executor.adb("command")` | Run ADB command |
| `executor.bash("command")` | Run bash command |
| `executor.execute("code")` | Run Python code |
| `executor.frida_connect()` | Connect to Frida |
| `executor.frida_attach("Gadget")` | Attach to process |
| `executor.frida_script(js_code)` | Load Frida script |
| `executor.frida_ssl_bypass()` | Bypass SSL pinning |
| `executor.frida_network_monitor()` | Monitor traffic |
| `executor.apk_patch("/path/app.apk")` | Patch with Gadget |
| `executor.apk_install("/path/app.apk")` | Install APK |
| `executor.apk_launch("com.example.app")` | Launch app |
| `executor.full_setup("https://url/app.apk")` | Complete setup |
| `executor.download("url", "/dest")` | Download file |
| `executor.ls("/path")` | List directory |
| `executor.read("/path/file")` | Read file |